In [1]:
import pandas as pd
import numpy as np
import datetime
import numba

In [2]:
import tqdm

Preprocess functions. Also can be imported from preprocess.py

In [3]:
# All preprocessing in one function
def preprocess_deals_data(data, vol_bins=None, vol_labels=None):
    '''
    Preprocess dataframe with deals
    Consolidate market orders, returns new dataframe
    Add features: Day of week, Hour, Time
    '''
    # aggregate to obtain market order instant impact
    # deals with the same time are caused by one market order
    data = data.groupby(by=['Time', 'AGGRESSOR_SIDE'], as_index=False) \
        .agg({'PRICE': ['min', 'max'], 'SIZE': 'sum', })
    
    # Flatten columns (!)
    data.columns = ['_'.join(z) if z[1]!='' else z[0] for z in data.columns]
    
    # Add datetime features
    data['Date'] = data['Time'].map(lambda d: d.date())  # only date
    data['TimeOnly'] = data['Time'].map(lambda d: d.time())  # only time
    data['DOW'] = data['Time'].map(lambda d: d.isoweekday())  # day of week
    data['H'] = data['Time'].map(lambda d: d.hour)  # hour (for filtering)
    
    # Impact calculation
    data['Impact'] = data['PRICE_max'] - data['PRICE_min']

    return data

In [4]:
@numba.njit()
def process_day(order_time, time, bid, ask):
    '''
    Function gets order time array (from deals file), and market bid, ask and time array (from LOB file)
    For each element in order time array it finds corresponding bid and ask
    Returns arrays of bids ands asks for market orders
    '''
    j = 0
    res_bid = np.zeros_like(order_time, dtype=np.float32)
    res_ask = np.zeros_like(order_time, dtype=np.float32)
    for i, t in enumerate(order_time):
        while time[j] < t:
            j += 1
        res_bid[i] = bid[j]
        res_ask[i] = ask[j]

    return res_bid, res_ask

In [5]:
def extract_day(dt, orders, LOB_folder, LOB_prefix):
    '''
    Function gets date, opens corresponding file with LOB history
    and return best bid, best ask and time arrays
    Also return order history time array from deals file (all in numpy format for numba)
    '''
    orders_time = orders[orders['Date'] == dt.date()]['Time'].to_numpy()
    
    fname = LOB_folder + LOB_prefix + dt.strftime('%m%d') + '.feather'  
    lob_df = pd.read_feather(fname)
    time_lob = lob_df['Time'].to_numpy()
    bid_lob = lob_df['BID_PRICE1'].to_numpy()
    ask_lob = lob_df['ASK_PRICE1'].to_numpy()
    return orders_time, time_lob, bid_lob, ask_lob

In [6]:
def date_range(start_date, end_date, only_workdays=False):
    '''
    Simple generator to obtain dates in defined date range
    '''
    for i in range(int((end_date - start_date).days) + 1):
        dt = start_date + datetime.timedelta(days=i)
        if only_workdays and dt.isoweekday() in [6, 7]:
            continue
        yield start_date + datetime.timedelta(days=i)

In [7]:
def add_lob_prices(market_orders, start_date, end_date, LOB_folder, LOB_prefix):
    '''
    Function processes market order dataframe and add bid and ask columns
    Returns new dataframe
    '''
    for dt in tqdm.tqdm_notebook(date_range(start_date, end_date, only_workdays=True), 
                                 total=int((end_date-start_date).days)+1):
        fname = LOB_folder + LOB_prefix + dt.strftime('%m%d') + '.feather'
        print(fname)
        try:
            lob_df = pd.read_feather(fname)
        except FileNotFoundError:
            print(f"{dt} skipped")
            continue

        if lob_df.shape[0] == 0:
            print(f"{fname} no quotes")
            continue

        order_time, lob_time, lob_bid, lob_ask = extract_day(dt, market_orders, LOB_folder, LOB_prefix)
        order_bid, order_ask = process_day(order_time, lob_time, lob_bid, lob_ask)

        market_orders.loc[market_orders['Date'] == dt.date(), 'BID'] = order_bid
        market_orders.loc[market_orders['Date'] == dt.date(), 'ASK'] = order_ask
    
    market_orders['MID'] = (market_orders['BID'] + market_orders['ASK']) / 2
    market_orders['Spread'] = np.round(market_orders['ASK'] - market_orders['BID'], decimals=4)
    
    return market_orders

## USDRUB data preprocessing

In [8]:
data = pd.read_feather('Data/USDRUB_TOM_trades.feather')  # load market order deals file

In [9]:
data[:5]

,Time,PRICE,SIZE,AGGRESSOR_SIDE,TRADE_VALUE
0,2021-02-01 10:00:00.271880,75.5975,100,B,7559750.0
1,2021-02-01 10:00:00.271880,75.6000,500,B,37800000.0
2,2021-02-01 10:00:00.271880,75.6700,63,B,4767210.0
3,2021-02-01 10:00:00.304507,75.6625,10,B,756625.0
4,2021-02-01 10:00:00.346729,75.6575,3,B,226972.5


In [10]:
market_orders = preprocess_deals_data(data)  # grouping and add features
market_orders[:5]

,Time,AGGRESSOR_SIDE,PRICE_min,PRICE_max,SIZE_sum,Date,TimeOnly,DOW,H,Impact
0,2021-02-01 10:00:00.271880,B,75.5975,75.6700,663,2021-02-01,10:00:00.271880,1,10,0.0725
1,2021-02-01 10:00:00.304507,B,75.6625,75.6625,10,2021-02-01,10:00:00.304507,1,10,0.0000
2,2021-02-01 10:00:00.346729,B,75.6575,75.6575,3,2021-02-01,10:00:00.346729,1,10,0.0000
3,2021-02-01 10:00:00.360369,S,75.6025,75.6025,20,2021-02-01,10:00:00.360369,1,10,0.0000
4,2021-02-01 10:00:00.377261,S,75.6025,75.6025,4,2021-02-01,10:00:00.377261,1,10,0.0000


In [11]:
print(market_orders['Time'].min())
print(market_orders['Time'].max())
print(market_orders.shape)

2021-02-01 10:00:00.271880
2021-10-06 23:49:57.338028
(4912952, 10)


In [12]:
# Add bid and ask prices for a moments of deals from LOB history files
LOB_folder = 'Data/LOB_USDRUB/'
LOB_prefix = 'LOB_'
start_date = datetime.datetime.strptime('2021-02-01', '%Y-%m-%d')
end_date = datetime.datetime.strptime('2021-10-06', '%Y-%m-%d')

In [13]:
market_orders = add_lob_prices(market_orders, start_date, end_date, LOB_folder, LOB_prefix)

C:\Users\Sharp.Ra\AppData\Local\Temp/ipykernel_19712/3349744981.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for dt in tqdm.tqdm_notebook(date_range(start_date, end_date, only_workdays=True),


  0%|          | 0/248 [00:00<?, ?it/s]

Data/LOB_USDRUB/LOB_0201.feather
Data/LOB_USDRUB/LOB_0202.feather
Data/LOB_USDRUB/LOB_0203.feather
Data/LOB_USDRUB/LOB_0204.feather
Data/LOB_USDRUB/LOB_0205.feather
Data/LOB_USDRUB/LOB_0208.feather
Data/LOB_USDRUB/LOB_0209.feather
Data/LOB_USDRUB/LOB_0210.feather
Data/LOB_USDRUB/LOB_0211.feather
Data/LOB_USDRUB/LOB_0212.feather
Data/LOB_USDRUB/LOB_0215.feather
Data/LOB_USDRUB/LOB_0216.feather
Data/LOB_USDRUB/LOB_0217.feather
Data/LOB_USDRUB/LOB_0218.feather
Data/LOB_USDRUB/LOB_0219.feather
Data/LOB_USDRUB/LOB_0222.feather
Data/LOB_USDRUB/LOB_0223.feather
Data/LOB_USDRUB/LOB_0223.feather no quotes
Data/LOB_USDRUB/LOB_0224.feather
Data/LOB_USDRUB/LOB_0225.feather
Data/LOB_USDRUB/LOB_0226.feather
Data/LOB_USDRUB/LOB_0301.feather
Data/LOB_USDRUB/LOB_0302.feather
Data/LOB_USDRUB/LOB_0303.feather
Data/LOB_USDRUB/LOB_0304.feather
Data/LOB_USDRUB/LOB_0305.feather
Data/LOB_USDRUB/LOB_0308.feather
Data/LOB_USDRUB/LOB_0308.feather no quotes
Data/LOB_USDRUB/LOB_0309.feather
Data/LOB_USDRUB/LOB_031

In [14]:
market_orders[:5]

,Time,AGGRESSOR_SIDE,PRICE_min,PRICE_max,SIZE_sum,Date,TimeOnly,DOW,H,Impact,BID,ASK,MID,Spread
0,2021-02-01 10:00:00.271880,B,75.5975,75.6700,663,2021-02-01,10:00:00.271880,1,10,0.0725,75.565002,75.599998,75.582504,0.0350
1,2021-02-01 10:00:00.304507,B,75.6625,75.6625,10,2021-02-01,10:00:00.304507,1,10,0.0000,75.567497,75.662498,75.614998,0.0950
2,2021-02-01 10:00:00.346729,B,75.6575,75.6575,3,2021-02-01,10:00:00.346729,1,10,0.0000,75.602501,75.657501,75.630005,0.0550
3,2021-02-01 10:00:00.360369,S,75.6025,75.6025,20,2021-02-01,10:00:00.360369,1,10,0.0000,75.602501,75.625000,75.613754,0.0225
4,2021-02-01 10:00:00.377261,S,75.6025,75.6025,4,2021-02-01,10:00:00.377261,1,10,0.0000,75.602501,75.639999,75.621246,0.0375


In [21]:
# Save results
market_orders.to_feather('market_orders_USDRUB.feather')

## EURRUB data preprocessing

In [22]:
data = pd.read_feather('Data/EURRUB_TOM_trades.feather')

In [23]:
data[:5]

,Time,PRICE,SIZE,AGGRESSOR_SIDE
0,2021-02-01 10:00:01.987427,91.7075,200,B
1,2021-02-01 10:00:01.987427,91.7075,293,B
2,2021-02-01 10:00:03.885514,91.7100,30,B
3,2021-02-01 10:00:04.614574,91.7000,1,S
4,2021-02-01 10:00:04.674547,91.5850,6,S


In [24]:
market_orders_eur = preprocess_deals_data(data)
market_orders_eur[:5]

,Time,AGGRESSOR_SIDE,PRICE_min,PRICE_max,SIZE_sum,Date,TimeOnly,DOW,H,Impact
0,2021-02-01 10:00:01.987427,B,91.7075,91.7075,493,2021-02-01,10:00:01.987427,1,10,0.0
1,2021-02-01 10:00:03.885514,B,91.7100,91.7100,30,2021-02-01,10:00:03.885514,1,10,0.0
2,2021-02-01 10:00:04.614574,S,91.7000,91.7000,1,2021-02-01,10:00:04.614574,1,10,0.0
3,2021-02-01 10:00:04.674547,S,91.5850,91.5850,6,2021-02-01,10:00:04.674547,1,10,0.0
4,2021-02-01 10:00:04.747542,B,91.8050,91.8050,3,2021-02-01,10:00:04.747542,1,10,0.0


In [25]:
print(market_orders_eur['Time'].min())
print(market_orders_eur['Time'].max())
print(market_orders_eur.shape)

2021-02-01 10:00:01.987427
2021-10-20 23:47:25.917780
(857010, 10)


In [26]:
market_orders_eur.to_feather('market_orders_EURRUB.feather')

Unfortunately, I have no LOB data to use this market order book in tests.